In [1]:
%load_ext autoreload
%autoreload 2

import os
import params
from utils import *
import numpy as np
import matplotlib.pyplot as plt
from time import sleep

from matplotlib.gridspec import GridSpec

## Cell 1: Load triggers times

In [8]:
[int(idx) for idx in recording_numbers.split()]

[11, 12]

In [11]:
"""
    Variables
    
    DO NOT CHANGE VALUES HERE UNLESS DEBUG/SPECIFIC USE
    
    All the variables used in this part of the cell should always refere to your 'params.py' file
    unless you want to manually change them only for this run (i.e. debugging). 
    You may have to add those variable into the function you want to adapt as only the minimal 
    amount of var are currently given to functions as inputs.
"""
#List of all the recording names
recording_names = params.recording_names

#Experiment name
exp = params.exp

#Analysis output directory
output_directory=params.output_directory

#Sampling rate of the mea
fs = params.fs  

#Trigger directory
triggers_directory= params.triggers_directory

"""
    Input
"""




cells_to_skip = []  ### LOAD HERE CELLS TO SKIP IF YOU WANT TO SELECT SPECIFICALY SOME OF THEM


stimulus_frequency = 1 # Change here
bin_size = 0.050 #s
nb_triggers_by_repetition = 4



print("All recordings : ")
#Find rec triggers
print(*['{} : {}'.format(i,recording_name) for i, recording_name in enumerate(recording_names)], sep="\n")
sleep(0.5)
recording_number = int(input("\nSelect recording BEFORE drugs: "))
rec_before = recording_names[recording_number]
print(f"\nSelected recording BEFORE drugs: {rec_before} \n")


# print(*['{} : {}'.format(i,recording_name) for i, recording_name in enumerate(recording_names)], sep="\n")
recording_numbers = input("\nSelect recording(s) AFTER drugs (if given numbers can be splited, multiple recs are taken): ")
recs_after = [recording_names[int(rec)] for rec in recording_numbers.split()]
print(f"\nSelected recording After drugs: {recs_after} \n \n")
nb_recs_after=len(recs_after)

Flicker_directory = os.path.normpath(os.path.join(output_directory,r'FullField_recs_{}>{}'.format(recording_number,[int(idx) for idx in recording_numbers.split()])))
if not os.path.isdir(Flicker_directory): os.makedirs(Flicker_directory)

    
    
"""
    Processing
"""

#load triggers
trig_data_before = load_obj(os.path.normpath(os.path.join(params.triggers_directory,'{}_{}_triggers.pkl'.format(exp,rec_before))))
stim_onsets_before = trig_data_before['indices']/params.fs 

print(f"Total triggers number rec BEFORE : {len(stim_onsets_before)}")

trig_data_after = [load_obj(os.path.normpath(os.path.join(params.triggers_directory,'{}_{}_triggers.pkl'.format(exp,rec)))) for rec in recs_after]
stim_onsets_after = [data['indices']/params.fs for data in trig_data_after]

print(f"\nTotal triggers number rec AFTER : {[len(stim_onsets) for stim_onsets in stim_onsets_after]}")

spike_trains=load_obj(os.path.join(output_directory, r'{}_fullexp_neurons_data.pkl'.format(exp)))

cells=list(spike_trains.keys())
spike_times={}
for cell in cells:
    if cell in cells_to_skip: continue
    spike_times[cell] = (spike_trains[cell][rec_before])

cells=list(spike_trains.keys())
spike_times_before={}
for cell in cells:
    if cell in cells_to_skip: continue
    spike_times_before[cell] = (spike_trains[cell][rec_before])

    
    
spike_times_after=[]    
for idx in range(len(recs_after)):
    rec_dict = {}
    for cell in cells:
        if cell in cells_to_skip: continue
        rec_dict[cell] = (spike_trains[cell][recs_after[idx]])
    spike_times_after.append(rec_dict)

print('Total : {} neurons loaded \n\nClusters id :\n{}\n'.format(len(spike_times.keys()),cells))

All recordings : 
0 : 00_Checkerboard_30ND50%_20pix30checks_30Hz
1 : 01_Checkerboard_30ND50%_16pix40checks_30Hz
2 : 02_DG_30ND50%_2sT_50Hz
3 : 03_Chirp_20reps_30ND50%_50Hz
4 : 04_Flicker_BeforeDrugs_30ND10%_1Hz
5 : 05_VDH_FF+Disk_GreyLevel_100µm_20reps_N8_Z(-25)_25ND50%_40Hz
6 : 06_VDH_FF+Disk_FullContrast_100µm(dark)_5reps_N8_Z(-30)_25ND50%_40Hz
7 : 07_VDH_FF_70reps_N8_Z(-30)_30ND50%_40Hz
8 : 08_Flicker_LAP4+ACET_t+10_30ND10%_1Hz
9 : 09_VDH_LAP4+ACET_FF+Disk_FullContrast_100µm(dark)_5reps_N8
10 : 10_HoloStim1_LAP4+ACET_N19_Z(-25)
11 : 11_OptoStim1_LAP4+ACET_15ND50%_1Hz
12 : 12_OptoStim1_LAP4+ACET_5ND50%_1Hz
13 : 13_OptoStim2_TPMPA_t10_5ND50%_1Hz
14 : 14_OptoStim2_TPMPA_t30_5ND50%_1Hz
15 : 15_HoloStim2_TPMPA_t30_N19_Z(-25)
16 : 16_OptoStim3_18BG(10uM)_t0-10_5ND50%_1Hz
17 : 17_HoloStim3_18BG(10uM)_t10_N19_Z(-25)
18 : 18_OptoStim4_18BG(20uM)_t0-10_5ND50%_1Hz
19 : 19_HoloStim4_18BG(20uM)_t10_N19_Z(-25)
20 : 20_HoloStim4_18BG(20uM)_t20_N19_Z(-25)

Select recording BEFORE drugs: 8

Selected

## Cell 2: Compute Raster and psths

In [12]:
################
#                /!\ CHANGE BELOW /!\
#                Change frequency, bin_size (if needed)
#                Change name of directory below
#                /!\ 3 changes in total /!\
################

analyse = {}

if nb_recs_after==1: rec_labels=['before', 'after']
if nb_recs_after==2: rec_labels=['before', 'after', 'after2']
if nb_recs_after==3: rec_labels=['before', 'after', 'after2', 'after3']

for recording in rec_labels:
        
    if recording=='before':
        onsets=stim_onsets_before
        spikes=spike_times_before
    if recording=='after':
        onsets=stim_onsets_after[0]
        spikes=spike_times_after[0]
    if recording=='after2':
        onsets=stim_onsets_after[1]
        spikes=spike_times_after[1]
    if recording=='after3':
        onsets=stim_onsets_after[2]
        spikes=spike_times_after[2]

    
    nb_triggers = len(onsets)

    sequence_first_indices = list(range(0,nb_triggers,nb_triggers_by_repetition))
    duration_repetition = nb_triggers_by_repetition / stimulus_frequency
    nb_bins = int(duration_repetition / bin_size)
#     print(sequence_first_indices, duration_repetition, nb_bins)
    
    for cell_idx, cell_nb in tqdm(enumerate(cells)):
        
        if not cell_nb in analyse.keys(): analyse[cell_nb] = {}
        
        SU_sptimes = spikes[cell_nb]

        # Flashes: Get the repeated sequence times for the specified position
        nb_repetitions = int(nb_triggers/nb_triggers_by_repetition)   
        repeated_sequences_times = []
        for i in sequence_first_indices:
            times = onsets[i:i+nb_triggers_by_repetition+1]
            repeated_sequences_times += [[times[0], times[-1]]]

        # Build the spike trains corresponding to stimulus repetitions
        spike_trains = []
        for i in range(len(repeated_sequences_times)):
#             spike_train = restrict_array(spikes, repeated_sequences_times[i][0], repeated_sequences_times[i][1])
            spike_train = SU_sptimes[(SU_sptimes >= repeated_sequences_times[i][0]) & (SU_sptimes <= repeated_sequences_times[i][1])]
            spike_trains += [spike_train]

        # Align the spike trains
        for i in range(len(spike_trains)):
            spike_trains[i] = spike_trains[i] - repeated_sequences_times[i][0]

        # Compute psth
        binned_spikes = np.empty((nb_repetitions,nb_bins))
        for i in range(nb_repetitions):
            binned_spikes[i,:] = np.histogram(spike_trains[i], bins=nb_bins, range=(0,duration_repetition))[0]

        # Compute sum
        binned_spikes = np.sum(binned_spikes, axis=0) 

        # Transform spike count in firing rate
        binned_spikes = binned_spikes / nb_repetitions /bin_size

#         analyse[cell_nb][recording_name]["repeated_sequences_times"] = repeated_sequences_times
        analyse[cell_nb][recording]={"spike_trains": spike_trains, "psth": binned_spikes}

np.save(os.path.join(Flicker_directory,f"data"), analyse)

print('--- Cell Done ---')

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

--- Cell Done ---


## Cell 3: Plotting

In [13]:
################
#             
#                Change frequency, bin_size (if needed)
#                Change labels
################

for cell in tqdm(cells[:]):
    fig = plt.figure(figsize=(10,10))
    gs = GridSpec(2+3*(nb_recs_after+1)+3, 6, figure=fig)
    for i in range(nb_recs_after+1+2):
        if i==0:
            #draw the stimulus
            ax = fig.add_subplot(gs[0:2,:])
            ON=list(np.zeros(int(nb_bins/4))+1)
            OFF=list(np.zeros(int(nb_bins/4)))
            vec=[0]+ON+OFF+ON+OFF
            ax.plot(vec, '-', color='k', lw=3)
            ax.set_ylabel('Stimulus', fontsize=15)
            ax.set_xticks([])
            ax.set_xlim([0,nb_bins])
            ax.set_title('Cell {}'.format(cell), fontsize=28)
            
        if i==1:
            #raster before
            ax = fig.add_subplot(gs[2:5,:])
            ax.eventplot(analyse[cell]['before']['spike_trains'])
            ax.set_xlim([0,4])
            ax.set_ylabel('Before', fontsize=25)
            ax.set_xticks([])
            for j in range(0,4+1,1):
                ax.axvline(j,ymin=0, ymax=1, ls='--',color='grey')
                
        if i==2:
            #raster after
            ax = fig.add_subplot(gs[5:8,:])
            ax.eventplot(analyse[cell]['after']['spike_trains'])
            ax.set_xlim([0,4])
            ax.set_ylabel('After', fontsize=20)
            ax.set_xticks([])
            for j in range(0,4+1,1):
                ax.axvline(j,ymin=0, ymax=1, ls='--',color='grey')
                
        if nb_recs_after>1 and i==3:
            #raster after 2
            ax = fig.add_subplot(gs[8:11,:])
            ax.eventplot(analyse[cell]['after2']['spike_trains'])
            ax.set_xlim([0,4])
            ax.set_ylabel('After 2', fontsize=20)
            ax.set_xticks([])
            for j in range(0,4+1,1):
                ax.axvline(j,ymin=0, ymax=1, ls='--',color='grey')
                
        if nb_recs_after>2 and i==4:
            #raster after 3
            ax = fig.add_subplot(gs[11:14,:])
            ax.eventplot(analyse[cell]['after3']['spike_trains'])
            ax.set_xlim([0,4])
            ax.set_ylabel('After 3', fontsize=20)
            ax.set_xticks([])
            for j in range(0,4+1,1):
                ax.axvline(j,ymin=0, ymax=1, ls='--',color='grey')
                
        if i==2+nb_recs_after:
            ax = fig.add_subplot(gs[2+3+nb_recs_after*3:2+3+nb_recs_after*3+3,:])
            ax.plot(analyse[cell]['before']['psth'], label='control_10^5R*')         # 'Control'
            ax.plot(analyse[cell]['after']['psth'], label='TPMPA_10^5R*')       #  'LAP4+ACET_t10' # OptoStim1_10^4R*
            if nb_recs_after>1: ax.plot(analyse[cell]['after2']['psth'], label='SR93351_10^5R*')  
            if nb_recs_after>2: ax.plot(analyse[cell]['after3']['psth'], label='')  
            
            ax.set_xlim([0,nb_bins])
            ax.set_xticks (range(0,nb_bins+1,int(nb_bins/8)))
            ax.set_xticklabels (np.arange(0,4.1,0.5))
            ax.set_xlabel('Seconds')
            ax.set_ylabel('Psths', fontsize=25)
            ax.legend()
            for j in range(0,nb_bins+1,int(nb_bins/4)):
                ax.axvline(j,ymin=0, ymax=1, ls='--',color='grey')
    
    plt.subplots_adjust(wspace=0, hspace=0)
    fig_file = os.path.join(Flicker_directory,f'Cell_{cell}.png')
    plt.savefig(fig_file, dpi=fig.dpi)
    plt.close()
print('--- Cell Done ---')

  0%|          | 0/212 [00:00<?, ?it/s]

--- Cell Done ---
